# The Graph Data Access

In this notebook, we retrieve live Rai system data from [The Graph](https://thegraph.com/explorer/subgraph/reflexer-labs/rai-mainnet?query=Example%20query) and save it as a CSV for analysis in the [Graph Analysis](TheGraphDataAnalysis.ipynb) notebook.

## Full Reflexer Labs dashboad query
```
{
  internalCoinBalances(where: {accountHandler: "${GEB_STABILITY_FEE_TREASURY}"}) { 
    balance
  }
  accountingEngine(id: "current") { # can it use block number?
    activeDebtAuctions
    activeSurplusAuctions
  }
  collateralType(id: "ETH-A") {
    currentPrice {
      value
    }
    currentMedianizerUpdate {
      value
    }
    debtCeiling
    activeLiquidations
    liquidationsStarted
    totalCollateral
    totalAnnualizedStabilityFee
  }
  systemState(id: "current") { # can it use block number?
    coinUniswapPair {
      reserve0
      reserve1
    }
    currentCoinMedianizerUpdate{
      value
    }
    currentRedemptionRate {
      eightHourlyRate
      annualizedRate
      hourlyRate
      createdAt
    }
    currentRedemptionPrice {
      value
    }
    erc20CoinTotalSupply
    globalDebt
    globalDebtCeiling
    totalActiveSafeCount
    coinAddress
    wethAddress
    systemSurplus
    debtAvailableToSettle
  }
  dailyStats(first: 1000, orderDirection: desc) {
    globalDebt
    timestamp
    redemptionRate {
      twentyFourHourlyRate
    }
  }
  hourlyStats(first: 1000, orderBy: timestamp, orderDirection: desc) {
    globalDebt
    timestamp
    redemptionRate {
      hourlyRate
    }
    redemptionPrice{
      value
    }
    marketPriceUsd
  }
}
```

In [1]:
# import libraries
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt

# Graph
url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'

## Hourly stats

In [2]:
query_header = '''
query {{
    hourlyStats(first: 1000, skip:{}) {{'''

query_tail = '''    
}
}'''

query_body = '''
 timestamp
    blockNumber
    redemptionRate {
      hourlyRate
    }
    redemptionPrice {
      id
      block
      value
      
    }
    marketPriceUsd # price of COIN in USD (uni pool price * ETH median price)
    marketPriceEth # Price of COIN in ETH (uni pool price)
    globalDebt
    erc20CoinTotalSupply
'''


n = 0
hourly = []
while True:
    print(f'request {n+1}')
    query = query_header.format(n*1000) + query_body + query_tail
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['hourlyStats']
    print(f'results {len(s)}')
    hourly.extend(s)
    print(f'total {len(hourly)}')
    n += 1
    if len(s) < 1000:
        break
        
hourlyStats = pd.DataFrame(hourly)

request 1
results 878
total 878


In [3]:
hourlyStats

,blockNumber,erc20CoinTotalSupply,globalDebt,marketPriceEth,marketPriceUsd,redemptionPrice,redemptionRate,timestamp
0,11857308,1988.440432971667032725,1988.577499489843641169436691184972,0.002589462210048543650949087456484599,4.670500166535429740591356399938391,"{'block': '11849022', 'id': '0xe313271188492c4...",{'hourlyRate': '1'},1613338681
1,11860793,2788.440432971667032725,2788.617717345781163523077653358556,0.002481355746535171105569862943787003,4.377049344386119132419461918241039,"{'block': '11860755', 'id': '0x40066774f6fb7d3...",{'hourlyRate': '1'},1613384616
2,11861040,2788.440432971667032725,2788.657935201718685876567866781341,0.002481355746535171105569862943787003,4.406833024856393396658172078078713,"{'block': '11861008', 'id': '0x0d551e2a2d55fa8...",{'hourlyRate': '1'},1613387890
3,11861337,2788.440432971667032725,2788.657935201718685876567866781341,0.002481355746535171105569862943787003,4.419265710853707004204445293455575,"{'block': '11861237', 'id': '0x5c67dbb323db039...",{'hourlyRate': '1'},1613392101
4,11861474,3048.440432971667032725,3048.645954953510904626288422086589,0.001768934955701819449591545208406114,3.194329891923853932595347583210722,"{'block': '11861237', 'id': '0x5c67dbb323db039...",{'hourlyRate': '1'},1613394096
...,...,...,...,...,...,...,...,...
873,12102553,42745404.758183382492966317,43002317.67842959364547956873089089,0.001715865353029436848271295650522285,2.949074940905223405492358547509156,"{'block': '12102288', 'id': '0xd599028d8a5266f...",{'hourlyRate': '1.000037926578715640400481731'},1616602156
874,12102891,42745404.758183382492966317,43002317.67842959364547956873089089,0.001719001469176058986497189981576783,2.939767552526129036348034418893351,"{'block': '12102858', 'id': '0xc23b42e7fa9a362...",{'hourlyRate': '1.000028355150965513546258103'},1616606316
875,12103189,42745404.758183382492966317,43002317.67842959364547956873089089,0.001722145111783343832447186851133831,2.906670962570163387281010911080703,"{'block': '12103118', 'id': '0x0c60f106db7bd31...",{'hourlyRate': '1.000028355150965513546258103'},1616610477
876,12103374,42745404.758183382492966317,43002475.22455560279908210492660422,0.001730278709549040165389546884191998,2.920399011551060971947785022076938,"{'block': '12103118', 'id': '0x0c60f106db7bd31...",{'hourlyRate': '1.000028355150965513546258103'},1616612864


In [4]:
hourlyStats.redemptionPrice.values[-1]

{'block': '12103424',
 'id': '0x872edab40fa2e69ef7ed0ef6b0eac0a1b8218bdbf9c91582d1fc3b021e0b5660-47',
 'value': '3.024816061988028774276896929'}

In [5]:
hourlyStats.redemptionRate.values[-1]

{'hourlyRate': '1.000028355150965513546258103'}

In [6]:
hourlyStats['hourlyredemptionPriceActual'] = hourlyStats.redemptionPrice.apply(lambda x: x['value'])
hourlyStats['hourlyredemptionRateActual'] = hourlyStats.redemptionRate.apply(lambda x: x['hourlyRate'])
hourlyStats['erc20CoinTotalSupply'] = hourlyStats['erc20CoinTotalSupply'].astype(float)
hourlyStats['globalDebt'] = hourlyStats['globalDebt'].astype(float)
hourlyStats['blockNumber'] = hourlyStats['blockNumber'].astype(int)
hourlyStats['hourlymarketPriceEth'] = hourlyStats['marketPriceEth'].astype(float)
del hourlyStats['marketPriceEth']
hourlyStats['hourlymarketPriceUsd'] = hourlyStats['marketPriceUsd'].astype(float)
del hourlyStats['marketPriceUsd']
hourlyStats['hourlyredemptionPriceActual'] = hourlyStats['hourlyredemptionPriceActual'].astype(float)
hourlyStats['hourlyredemptionRateActual'] = hourlyStats['hourlyredemptionPriceActual'].astype(float)

In [7]:
hourlyStats.describe()

,blockNumber,erc20CoinTotalSupply,globalDebt,hourlyredemptionPriceActual,hourlyredemptionRateActual,hourlymarketPriceEth,hourlymarketPriceUsd
count,8.780000e+02,8.780000e+02,8.780000e+02,878.000000,878.000000,878.000000,878.000000
mean,1.198084e+07,3.354361e+07,3.398548e+07,3.064162,3.064162,0.001878,3.209433
std,6.974728e+04,1.408364e+07,1.420170e+07,0.049037,0.049037,0.000296,0.440628
min,1.185731e+07,1.988440e+03,1.988577e+03,3.013686,3.013686,0.001600,2.850455
25%,1.192067e+07,2.893896e+07,2.923238e+07,3.017978,3.017978,0.001671,3.002190
50%,1.198031e+07,3.492160e+07,3.563961e+07,3.043398,3.043398,0.001772,3.132480
75%,1.204066e+07,4.484609e+07,4.506986e+07,3.113550,3.113550,0.002049,3.262714
max,1.210370e+07,4.899876e+07,4.918825e+07,3.140000,3.140000,0.004474,8.055382


## Daily stats - WIP

## System State 

In [8]:
blocknumbers = hourlyStats.blockNumber.values.tolist()
state = []
for i in blocknumbers:
    query = '''
    {
      systemState(block: {number:%s},id:"current") { 
        coinUniswapPair {
          reserve0
          reserve1
        }
        currentCoinMedianizerUpdate{
          value
        }
        currentRedemptionRate {
          eightHourlyRate
          annualizedRate
          hourlyRate
          createdAt
        }
        currentRedemptionPrice {
          value
        }
        erc20CoinTotalSupply
        globalDebt
        globalDebtCeiling
        totalActiveSafeCount
        coinAddress
        wethAddress
        systemSurplus
        debtAvailableToSettle
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['systemState']
    state.append(s)
        
systemState = pd.DataFrame(state)

systemState.head()
    

,coinAddress,coinUniswapPair,currentCoinMedianizerUpdate,currentRedemptionPrice,currentRedemptionRate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,wethAddress
0,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '661.764842786150958141', 'reserv...",None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.522357955470745187436691184972,115792089237316195423570985008687.9,0.06853325908830422221834559248594176,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
1,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
2,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
3,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
4,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,0.1161617985274885650688366678947734,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2


In [9]:
systemState['blockNumber'] = blocknumbers

In [10]:
systemState.coinUniswapPair.values[-1]

{'reserve0': '36665948.985869133181430285',
 'reserve1': '65179.862939851702594016'}

In [11]:
systemState.currentCoinMedianizerUpdate.values[-1]

{'value': '2.936512508917401291'}

In [12]:
systemState.currentRedemptionRate.values[-1]

{'annualizedRate': '1.28195672354123148495372313',
 'createdAt': '1616605864',
 'eightHourlyRate': '1.000226863721409254858091628',
 'hourlyRate': '1.000028355150965513546258103'}

In [13]:
systemState.currentRedemptionPrice.values[-1]

{'value': '3.024816061988028774276896929'}

In [14]:
systemState.columns

Index(['coinAddress', 'coinUniswapPair', 'currentCoinMedianizerUpdate',
       'currentRedemptionPrice', 'currentRedemptionRate',
       'debtAvailableToSettle', 'erc20CoinTotalSupply', 'globalDebt',
       'globalDebtCeiling', 'systemSurplus', 'totalActiveSafeCount',
       'wethAddress', 'blockNumber'],
      dtype='object')

In [15]:
systemState['RedemptionRateAnnualizedRate'] = systemState.currentRedemptionRate.apply(lambda x: x['annualizedRate'])
systemState['RedemptionRateHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['hourlyRate'])
systemState['RedemptionRateEightHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['eightHourlyRate'])
systemState['RedemptionPrice'] = systemState.currentRedemptionPrice.apply(lambda x: x['value'])
del systemState['currentRedemptionRate']
del systemState['currentRedemptionPrice']
systemState['RedemptionRateAnnualizedRate'] = systemState['RedemptionRateAnnualizedRate'].astype(float)
systemState['RedemptionRateHourlyRate'] = systemState['RedemptionRateHourlyRate'].astype(float)
systemState['RedemptionRateEightHourlyRate'] = systemState['RedemptionRateEightHourlyRate'].astype(float)
systemState['RedemptionPrice'] = systemState['RedemptionPrice'].astype(float)


In [16]:
systemState.head()

,coinAddress,coinUniswapPair,currentCoinMedianizerUpdate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,wethAddress,blockNumber,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice
0,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '661.764842786150958141', 'reserv...",None,0,2788.440432971667032725,2788.522357955470745187436691184972,115792089237316195423570985008687.9,0.06853325908830422221834559248594176,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11857308,1.0,1.0,1.0,3.14
1,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11860793,1.0,1.0,1.0,3.14
2,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861040,1.0,1.0,1.0,3.14
3,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861337,1.0,1.0,1.0,3.14
4,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",{'value': '0'},0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,0.1161617985274885650688366678947734,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861474,1.0,1.0,1.0,3.14


## Internal Coin Balances - WIP

## Accounting Engine - WIP

## Eth Price

In [17]:
blocknumbers = hourlyStats.blockNumber.values.tolist()
eth_price = []
for i in blocknumbers:
    query = '''
    {
      collateralType(block: {number:%s}, id: "ETH-A") {
        currentFsmUpdate {
          value
        }
        currentMedianizerUpdate {
          value
        }
        debtCeiling
        activeLiquidations
        liquidationsStarted
        totalCollateral
        totalAnnualizedStabilityFee
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['collateralType']
    eth_price.append(s)
        
collateralType = pd.DataFrame(eth_price)

collateralType['blockNumber'] = blocknumbers
collateralType.head()
    

,activeLiquidations,currentFsmUpdate,currentMedianizerUpdate,debtCeiling,liquidationsStarted,totalAnnualizedStabilityFee,totalCollateral,blockNumber
0,0,{'value': '1803.65643044'},{'value': '1803.65643044'},10000000,0,1.019999999999999999972831879,14.892360473369843164,11857308
1,0,{'value': '1803.65643044'},{'value': '1763.97493608'},10000000,0,1.019999999999999999972831879,14.892360473369843164,11860793
2,0,{'value': '1763.97493608'},{'value': '1775.97792296'},10000000,0,1.019999999999999999972831879,14.892360473369843164,11861040
3,0,{'value': '1763.97493608'},{'value': '1780.98836373'},10000000,0,1.019999999999999999972831879,14.892360473369843164,11861337
4,0,{'value': '1763.97493608'},{'value': '1805.79273513'},10000000,0,1.019999999999999999972831879,16.892360473369843164,11861474


In [18]:

collateralType['Eth_price'] = collateralType.currentFsmUpdate.apply(lambda x: x['value'])
collateralType['Eth_price'] = collateralType['Eth_price'].astype(float)


In [19]:
hourlyStats = hourlyStats.merge(collateralType,how='inner',on='blockNumber')

# remove duplicate values
del systemState['erc20CoinTotalSupply']
del systemState['globalDebt']

hourlyStats = hourlyStats.merge(systemState,how='inner',on='blockNumber')

In [20]:
hourlyStats.to_csv('saved_results/RaiLiveGraphData.csv')

## Safes

## Conclusion

Using The Graph, a lot of data about the Rai system can be obtained for analyzing the health of the system. With some data manipulation, these data streams could be intergrated into the Rai cadCAD model to turn it into a true decision support system.